### Generación de texto 

En este cuaderno, implementamos el modelo Transformer-XL de generación de texto utilizando PyTorch-Transformers: [Hugging Face](https://huggingface.co/).
Una de las caracteristicas de esta biblioteca es:

* Modelos pre-entrenados: Proporciona modelos pre-entrenados de arquitecturas de NLP de última generación y pesos pre-entrenados para varias variaciones de estos modelos.
* Preprocesamiento y API de fine-tuning: PyTorch-Transformers no se detiene en pesos pre-entrenados. También proporciona una API simple para realizar todos los pasos de preprocesamiento y ajuste necesarios para estos modelos. 
* Scripts de uso: también viene con scripts para ejecutar estos modelos y conjuntos de datos de referencia  como SQUAD 2.0 (Stanford Question Answering Dataset) y GLUE(General Language Understanding Evaluation). Al utilizar PyTorch-Transformers, se puede ejecutar directamente el modelo con estos conjuntos de datos y evaluar el rendimiento.
* Multilingüe: PyTorch-Transformers tiene soporte multilingüe. Esto se debe a que algunos de los modelos ya funcionan bien para varios lenguajes.
* Compatibilidad con TensorFlow: se puede importar checkpoints de TensorFlow como modelos en PyTorch
* Investigación: hay un campo de estudio creciente relacionado con la investigación del funcionamiento interno de transformadores a gran escala como BERT-

Usamos un modelo BERT preentrenado para este cuaderno.

In [8]:
!pip install pytorch-pretrained-bert


[notice] A new release of pip available: 23.0 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [9]:
from functools import partial
from tqdm import trange
import torch
import torch.nn.functional as F
import numpy as np
import pytorch_pretrained_bert
from pytorch_pretrained_bert import GPT2LMHeadModel, GPT2Tokenizer
for mod in (np, torch, pytorch_pretrained_bert):
    print(f'{mod.__name__}: {mod.__version__}')

numpy: 1.21.5
torch: 1.10.2
pytorch_pretrained_bert: 0.6.2


### Contruyamos el modelo  GPT2

OpenAI creó el modelo de lenguaje llamado GPT-2. GPT-2 es un modelo de lenguaje generativo basado en transformers que se entrenó en 40 GB de texto de Internet.

Al ser entrenado de manera no supervisada, simplemente aprende a predecir una secuencia de tokens más probables (es decir, palabras) que siguen un mensaje determinado, en función de los patrones que aprendió a reconocer a través de su entrenamiento.


In [10]:
seed = 0
np.random.seed(seed)
torch.random.manual_seed(seed)
torch.cuda.manual_seed(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

nombre_modelo = 'gpt2'
enc = GPT2Tokenizer.from_pretrained('gpt2')

modelo = GPT2LMHeadModel.from_pretrained(nombre_modelo)
modelo.to(device)
modelo.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (h): ModuleList(
      (0): Block(
        (ln_1): BertLayerNorm()
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
        )
        (ln_2): BertLayerNorm()
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
        )
      )
      (1): Block(
        (ln_1): BertLayerNorm()
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
        )
        (ln_2): BertLayerNorm()
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
        )
      )
      (2): Block(
        (ln_1): BertLayerNorm()
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
        )
        (ln_2): BertLayerNorm()
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
        )
      )
      (3): Block(
        (ln_1): BertLayerNorm()
      

Eliminamos los warnings de python y realizamos una prueba del funcionamiento del modelo.

In [11]:
import warnings
warnings.filterwarnings("ignore")

linea = "Proof"
tokens_tensor = torch.tensor(enc.encode(linea)).reshape(1, -1)
predicciones, _ = modelo(tokens_tensor)
tam_vocab = predicciones.shape[-1]
assert tam_vocab == 50257

In [12]:
predicciones.shape, tokens_tensor.shape

(torch.Size([1, 1, 50257]), torch.Size([1, 1]))

### Un ejemplo

In [13]:
seed = 0
prompt = "Cars were invented in"
max_predicciones = 16
top_k = 40

np.random.seed(seed)
torch.random.manual_seed(seed)
context_tokens = enc.encode(prompt)
context_tensor = torch.tensor(context_tokens, dtype=torch.long).reshape(1, -1)

prev = context_tensor
salida = context_tensor.clone()
past = None
for i in range(max_predicciones):
    predicciones, past = modelo(prev, past=past)
    context_size = prev.shape[1]
    assert predicciones.shape == (1, context_size, tam_vocab)
    
    last_prediction = predicciones[:, -1, :]
    topk = torch.topk(last_prediction, 10)   
    log_probs = F.softmax(topk.values, dim=-1)  # softmax among the top-k
    rand_idx_in_topk = torch.multinomial(log_probs, num_samples=1)
    predicted_index = topk.indices[0][rand_idx_in_topk]      
    
    prev = torch.tensor([[ predicted_index ]])
    salida = torch.cat((salida, prev), dim=-1)
    
print(enc.decode(salida[0].tolist()))

Cars were invented in 1789 by Thomas Jefferson's brother William Jefferson (1787-1809),


Construyamos un modelo de finalización de oraciones usando GPT-2. Intentaremos predecir la siguiente palabra en la oración: what is the fastest car in the _________

Ejemplo desde: [https://www.analyticsvidhya.com/blog/2019/07/pytorch-transformers-nlp-python/](https://www.analyticsvidhya.com/blog/2019/07/pytorch-transformers-nlp-python/)




In [14]:
texto = "What is the fastest car in the"
indexed_tokens = enc.encode(texto)

tokens_tensor = torch.tensor([indexed_tokens])
modelo = GPT2LMHeadModel.from_pretrained('gpt2')
modelo.eval()

# predicciones de tokens
with torch.no_grad():
  salidas = modelo(tokens_tensor)
  predicciones = salidas[0]

predicted_index = torch.argmax(predicciones[0, -1, :]).item()
predicted_text = enc.decode(indexed_tokens + [predicted_index])

# Predecimos la palabra predicha
print(predicted_text)

What is the fastest car in the world
